# Part 3: Single qubit gates using minimum duration $X$ and $Z$ axis rotations

### Input:

A string in the form of a comma-separated list of sequential rotations on one particular qubit.
The string takes the form: “{Axis}({Angle}}, {Axis}({Angle}), …” where {Axis} can be X or Y
and angle is an integer specifying the angle of rotation in degrees.

The durations of X and Z pulses in nanoseconds.

### Required Output:

Optimise the instruction sequence such that the total time taken is minimised using $X$ and $Z$ pulses.

### Solution:

#### Intuition: 

From part-2, we parameterized$^1$ U as Z$\left(\alpha\right)$X$\left(\beta\right)$Z$\left(\gamma\right)$ where

$$
\alpha=\tan^{-1}\left(\frac{-x_2}{x_1}\right)+ \tan^{-1}\left(\frac{x_3}{x_4}\right)\\
\gamma=\tan^{-1}\left(\frac{-x_2}{x_1}\right)- \tan^{-1}\left(\frac{x_3}{x_4}\right)\\
\beta=2\tan^{-1}\left(\frac{x_3 \sin\left(\frac{\alpha+\gamma}{2}\right)}{x_2 \sin\left(\frac{\alpha-\gamma}{2}\right)}\right)
$$

Since we wish to optimize the duration, $X$ pulse may require less time than $Z$. In this case, we should decmpose U as X$\left(\alpha\right)$Z$\left(\beta\right)$X$\left(\gamma\right)$. The single qubit unitary U in this case can be written as

$$U=\mathrm{X}\left(\alpha\right)\mathrm{Z}\left(\beta\right)\mathrm{X}\left(\gamma\right)=\begin{pmatrix}
\cos\left(\frac{\alpha+\gamma}{2}\right)\cos\left(\frac{\beta}{2}\right)+\dot{\iota} \cos\left(\frac{\alpha-\gamma}{2}\right)\sin\left(\frac{\beta}{2}\right)& 
-\sin\left(\frac{\alpha-\gamma}{2}\right)\sin\left(\frac{\beta}{2}\right)-\dot{\iota} \sin\left(\frac{\alpha+\gamma}{2}\right)\cos\left(\frac{\beta}{2}\right)\\
\sin\left(\frac{\alpha-\gamma}{2}\right)\sin\left(\frac{\beta}{2}\right)-\dot{\iota} \sin\left(\frac{\alpha+\gamma}{2}\right)\cos\left(\frac{\beta}{2}\right) &
\cos\left(\frac{\alpha+\gamma}{2}\right)\cos\left(\frac{\beta}{2}\right)-\dot{\iota} \cos\left(\frac{\alpha-\gamma}{2}\right)\sin\left(\frac{\beta}{2}\right)
\end{pmatrix}
$$

where 

$$\mathrm{X}\left(\theta\right)=\begin{pmatrix}
\cos \left(\frac{\theta}{2}\right) & -\dot{\iota} \sin \left(\frac{\theta}{2}\right)\\
-\dot{\iota} \sin \left(\frac{\theta}{2}\right) & \cos \left(\frac{\theta}{2}\right)
\end{pmatrix},
$$
and
$$
\mathrm{Z}\left(\theta\right)=\begin{pmatrix}
e^{-\dot{\iota}\left(\frac{\theta}{2}\right)} & 0\\
0 & e^{\dot{\iota}\left(\frac{\theta}{2}\right)}
\end{pmatrix}.
$$

Any general single qubit unitary belongs to SU(2) and thus can be represented as$^{2}$

$$U=\begin{pmatrix}
a & b\\
-\bar{b} & \bar{a}
\end{pmatrix} = \begin{pmatrix}
x_1+\dot{\iota}x_2 & x_3+\dot{\iota}x_4\\
-x_3+\dot{\iota}x_4 & x_1-\dot{\iota}x_2
\end{pmatrix}
$$

Therefore,

$$
x_1=\cos\left(\frac{\alpha+\gamma}{2}\right)\cos\left(\frac{\beta}{2}\right)\\
x_2=\cos\left(\frac{\alpha-\gamma}{2}\right)\sin\left(\frac{\beta}{2}\right)\\
x_3=-\sin\left(\frac{\alpha-\gamma}{2}\right)\sin\left(\frac{\beta}{2}\right)\\
x_4=-\sin\left(\frac{\alpha+\gamma}{2}\right)\cos\left(\frac{\beta}{2}\right)
$$

Solving this, we get

$$
\alpha=\tan^{-1}\left(\frac{-x_4}{x_1}\right)+ \tan^{-1}\left(\frac{x_3}{x_2}\right)\\
\gamma=\tan^{-1}\left(\frac{-x_4}{x_1}\right)- \tan^{-1}\left(\frac{x_3}{x_2}\right)\\
\beta=2\tan^{-1}\left(\frac{x_3 \sin\left(\frac{\alpha-\gamma}{2}\right)}{x_1 \cos\left(\frac{\alpha+\gamma}{2}\right)}\right)
$$

$^1$ https://qiskit.org/documentation/stubs/qiskit.quantum_info.OneQubitEulerDecomposer.html 

$^2$ https://lsa.umich.edu/content/dam/math-assets/math-document/reu-documents/REUsummer2015LiangThompsonDamelin.pdf

#### Algorithm Steps

Step-1: First we prepare the sequence of pulses either provided by user or chosen at random and the duration of gates X and Z.

Step-2: We then define the rotation operators for X$\left(\theta\right)$ and Z$\left(\theta\right)$ which will take the arguments $\theta$ from sequence. 

Step-3: From the intuition discussed above, we then merge our sequence into a single untiary.

Step-4: We obtain the arguments $\alpha,\beta,$ and $\gamma$ for the optimal pulse sequence  $\mathrm{Z}\left(\alpha\right)\mathrm{X}\left(\beta\right)\mathrm{Z}\left(\gamma\right)$ if duration of Z-gate is smaller or $\mathrm{X}\left(\alpha\right)\mathrm{Z}\left(\beta\right)\mathrm{X}\left(\gamma\right)$ if duration of X-gate is smaller.

Step 5: Then we check that our pulse sequence equates unitary U.

Step-6: Finally we print the optimal list of sequences.

#### Python Program

Step-1: First we prepare the sequence of pulses either provided by user or chosen at random.

In [2]:
import numpy as np
import random as rd
from Part3_function_file import *

while 1:
    choice = input('Enter pulse sequence manually: (y/n) ')
    if choice not in ['y', 'Y', 'n', 'N']:
        print('Incorrect choice')
    else:
        list_of_rotation_type=[]
        list_of_rotation_angle=[]
        if choice in ['y', 'Y']:
            sequence_input = input('Enter sequence of X and Y rotations separated by commas: Axis(Angle),Axis(Angle),... : ')
            for individual_pulse_in_sequence in sequence_input.split(','): #parsing the sequence for validity 
                if individual_pulse_in_sequence.split('(')[0] not in ['X', 'Y']:
                    raise Exception('Can only input X and Y pulse rotations')
                else:
                    list_of_rotation_type.append(individual_pulse_in_sequence.split('(')[0])  
                    
                if individual_pulse_in_sequence.split('(')[1][:-1].replace('.','',1).isnumeric() == False:
                    raise Exception('Incorrect input sequence')
                else:
                    list_of_rotation_angle.append(float(individual_pulse_in_sequence.split('(')[1][:-1]))
                    

        else:
            theta_min=0
            theta_max=360 
            list_of_rotation_type=[]
            list_of_rotation_angle=[]
            sequence_length = input('Enter length of sequence:')
            if sequence_length.isdigit() and sequence_length > '0':
                for _ in range(int(sequence_length)):
                    individual_pulse_type=rd.choice(['X','Y'])
                    list_of_rotation_type.append(individual_pulse_type)
                    individual_pulse_angle = rd.randint(theta_min, theta_max)
                    list_of_rotation_angle.append(individual_pulse_angle)
            else: 
                raise Exception('Not valid length')
        length_Z = float(input('Enter duration of Z gate in ns:'))
        length_X = float(input('Enter duration of X gate in ns:'))

        mainfunc(list_of_rotation_type,list_of_rotation_angle,length_Z,length_X)
        break

Enter pulse sequence manually: (y/n) y
Enter sequence of X and Y rotations separated by commas: Axis(Angle),Axis(Angle),... : X(43),Y(22),X(13),Y(29),X(43),Y(220),X(403),Y(202)
Enter duration of Z gate in ns:44
Enter duration of X gate in ns:45

 The overall rotation using input pulse sequence is: 

[[ 0.50754764-0.52621607j -0.57619456-0.36536539j]
 [ 0.57619456-0.36536539j  0.50754764+0.52621607j]]

 The overall rotation using optimal pulse sequence is the same: 

[[ 0.50754764-0.52621607j -0.57619456-0.36536539j]
 [ 0.57619456-0.36536539j  0.50754764+0.52621607j]]

 And the optimal pulse sequence is:

Z (  103.6557607060568  ) X ( 86.04253289908777 ) Z ( -11.586611679226886 )
